## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import os
import numpy as np
from sympy import Matrix
import pytest
from attr import define, field
from sympy import MutableDenseNDimArray as Arr


from zoomy_core.fvm.solver_numpy import Settings
from zoomy_core.model.basemodel import Model
import zoomy_core.model.initial_conditions as IC
import zoomy_core.model.boundary_conditions as BC
from zoomy_core.misc.misc import Zstruct, ZArray
import zoomy_core.misc.misc as misc
import zoomy_firedrake.firedrake_solver as dg


In [2]:
@define(frozen=True, slots=True, kw_only=True)
class SWE(Model):
    dimension: int = 2
    variables: Zstruct = field(init=False)
    aux_variables: Zstruct = field(default=1)
    _default_parameters: dict = field(
        init=False, factory=lambda: {"g": 9.81, "ex": 0.0, "ey": 0.0, "ez": 1.0}
    )
    
    def __attrs_post_init__(self):
        object.__setattr__(self, "variables", self.dimension + 2)
        super().__attrs_post_init__()

    def project_2d_to_3d(self):
        out = ZArray.zeros(6)
        dim = self.dimension
        z = self.position[2]
        b = self.aux_variables[0]
        h = self.variables[1]
        U = [hu / h for hu in self.variables[2 : 2 + dim]]
        rho_w = 1000.0
        g = 9.81
        out[0] = b
        out[1] = h
        out[2] = U[0]
        out[3] = 0 if dim == 1 else U[1]
        out[4] = 0
        out[5] = rho_w * g * h * (1 - z)
        return out

    def flux(self):
        dim = self.dimension
        b = self.aux_variables[0]
        h = self.variables[1]
        U = Matrix([hu / h for hu in self.variables[2 : 2 + dim]])
        g = self.parameters.g
        I = Matrix.eye(dim)
        F = Matrix.zeros(self.variables.length(), dim)
        F[1, :] = h * U.T
        F[2:, :] = h * U * U.T + g / 2 * h**2 * I
        return ZArray(F)


# Transformation to UFL Code (Medium)

### Map from Sympy to UFL

In [3]:
bcs = BC.BoundaryConditions(
    [
        BC.Extrapolation(tag="top"),
        BC.Extrapolation(tag="bottom"),
        BC.Extrapolation(tag="left"),
        BC.Extrapolation(tag="right"),
        # BC.Wall(tag="top"),
        # BC.Wall(tag="bottom"),
        # BC.Wall(tag="left"),
        # BC.Wall(tag="right"),
    ]
)

 ### Initial condition
def ic_q(x):
    R = 0.15
    r = np.sqrt((x[0] - 0.7)**2 + (x[1] - 0.7)**2)
    b = 0.1*np.sqrt((x[0] - 3.)**2 + (x[1] - 3.)**2)
    return np.array([0.*x[0], np.where(r <= R, 1., 0.9), 0.*x[0], 0.*x[0]])

ic = IC.UserFunction(ic_q)

model = SWE(
    dimension=2,
    boundary_conditions=bcs,
    initial_conditions=ic,
)

settings = Settings(name="Firedrake", output=Zstruct(directory="outputs/firedrake", filename='dg'))


2025-11-11 06:39:21.410 | WARNING  | zoomy_core.misc.misc:__init__:149 - No 'clean_directory' attribute found in output Zstruct. Default: False


In [4]:
solver = dg.FiredrakeHyperbolicSolver(settings=settings, time_end = 10.)

In [5]:
main_dir = misc.get_main_directory()
path_to_mesh = os.path.join(main_dir, "meshes", "channel_quad_2d", "mesh_coarse.msh")
solver.solve(path_to_mesh, model)

t = 0.003, dt = 2.915e-03
t = 0.013, dt = 9.757e-03
t = 0.023, dt = 1.072e-02
t = 0.035, dt = 1.166e-02
t = 0.047, dt = 1.199e-02
t = 0.060, dt = 1.260e-02
t = 0.073, dt = 1.286e-02
t = 0.086, dt = 1.325e-02
t = 0.099, dt = 1.349e-02
t = 0.113, dt = 1.375e-02
t = 0.127, dt = 1.397e-02
t = 0.141, dt = 1.415e-02
t = 0.155, dt = 1.435e-02
t = 0.170, dt = 1.448e-02
t = 0.185, dt = 1.465e-02
t = 0.199, dt = 1.475e-02
t = 0.214, dt = 1.491e-02
t = 0.229, dt = 1.498e-02
t = 0.244, dt = 1.512e-02
t = 0.260, dt = 1.517e-02
t = 0.275, dt = 1.529e-02
t = 0.290, dt = 1.534e-02
t = 0.306, dt = 1.539e-02
t = 0.321, dt = 1.540e-02
t = 0.336, dt = 1.541e-02
t = 0.352, dt = 1.542e-02
t = 0.367, dt = 1.543e-02
t = 0.383, dt = 1.543e-02
t = 0.398, dt = 1.544e-02
t = 0.413, dt = 1.544e-02
t = 0.429, dt = 1.545e-02
t = 0.444, dt = 1.546e-02
t = 0.460, dt = 1.546e-02
t = 0.475, dt = 1.547e-02
t = 0.491, dt = 1.547e-02
t = 0.506, dt = 1.548e-02
t = 0.522, dt = 1.548e-02
t = 0.537, dt = 1.549e-02
t = 0.553, d